# Projeto Final

Carolina Abdu e Mariana Meirelles

## Preparo do dataframe

Carregar e eliminar linhas com valores negativos

In [1]:
import pandas as pd
df = pd.read_csv("/ndt_tests_corrigido.csv")
display(df)


FileNotFoundError: [Errno 2] No such file or directory: '/ndt_tests_corrigido.csv'

In [ ]:
 df_treated = df[(df.select_dtypes(include='number') >= 0).all(axis=1)]
 display(df_treated)


## Métricas básicas

A seguir estão apresentados daataframes com as seguintes métricas por cliente e servidor:
- Média
- Mediana
- Variância
- Desvio padrão
- Quantis

In [ ]:
# Média por cliente e servidor
df_media = df_treated.groupby(['client', 'server'])[[
    'download_throughput_bps',
    'rtt_download_sec',
    'upload_throughput_bps',
    'rtt_upload_sec',
    'packet_loss_percent'
]].mean()


display(df_media)

In [ ]:
# Mediana pro cliente e servidor
df_mediana = df_treated.groupby(['client', 'server'])[[
    'download_throughput_bps',
    'rtt_download_sec',
    'upload_throughput_bps',
    'rtt_upload_sec',
    'packet_loss_percent'
]].median()

display(df_mediana)

In [ ]:
# Variância pro cliente e servidor
df_var= df_treated.groupby(['client', 'server'])[[
    'download_throughput_bps',
    'rtt_download_sec',
    'upload_throughput_bps',
    'rtt_upload_sec',
    'packet_loss_percent'
]].var()

display(df_var)

In [ ]:
# Desvio padrão pro cliente e servidor
df_desvio= df_treated.groupby(['client', 'server'])[[
    'download_throughput_bps',
    'rtt_download_sec',
    'upload_throughput_bps',
    'rtt_upload_sec',
    'packet_loss_percent'
]].std()

display(df_desvio)

NameError: name 'df_treated' is not defined

In [ ]:
#Percentis : 50, 90 e 99 %

# agrupando por cliente e servidor
df_quantis = df_treated.groupby(['client', 'server']).agg({
    'download_throughput_bps': ['quantile', lambda x: x.quantile(0.9), lambda x: x.quantile(0.99)],
    'upload_throughput_bps': ['quantile', lambda x: x.quantile(0.9), lambda x: x.quantile(0.99)],
    'rtt_download_sec': ['quantile', lambda x: x.quantile(0.9), lambda x: x.quantile(0.99)],
    'rtt_upload_sec': ['quantile', lambda x: x.quantile(0.9), lambda x: x.quantile(0.99)],
    'packet_loss_percent': ['quantile', lambda x: x.quantile(0.9), lambda x: x.quantile(0.99)]
})

display(df_quantis)

In [ ]:
df_treated.unique()

NameError: name 'df_treated' is not defined

### Gráficos

In [ ]:
#Escolha dos clienets que serão comparados
df_treated.groupby('client')[[
    'download_throughput_bps',
    'upload_throughput_bps',
    'rtt_download_sec',
    'rtt_upload_sec',
    'packet_loss_percent'
]].mean().reset_index()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Histograma
sns.histplot(df[df['client']=='client12']['packet_loss_percent'], bins=200, kde=True)
plt.title(f'Histograma do Throughput - Cliente client12')



NameError: name 'df' is not defined

In [ ]:
sns.histplot(df[df['client']=='client12']['packet_loss_percent'], bins=200, kde=True)
plt.title(f'Histograma do Throughput - Cliente client12')


In [ ]:
# Boxplot comparativo
sns.boxplot(x='client', y='packet_loss_percent', data=df[df['client'].isin(['client12', 'client05'])])

In [ ]:
# Scatter plot
sns.scatterplot(x='packet_loss_percent', y='upload_throughput_bps', hue='client', data=df[df['client'].isin(['client12', 'client05'])])